# Differentiable bitonic sort

[Bitonic sorts](https://en.wikipedia.org/wiki/Bitonic_sorter) allow creation of sorting networks with a sequence of fixed conditional swapping operations executed in parallel. A sorting network implements  a map from $\mathbb{R}^n \rightarrow \mathbb{R}^n$, where $n=2^k$ (sorting networks for non-power-of-2 sizes are possible but not trickier).

<img src="BitonicSort1.svg.png">

*[Image: from Wikipedia, by user Bitonic, CC0](https://en.wikipedia.org/wiki/Bitonic_sorter#/media/File:BitonicSort1.svg)*

The sorting network for $n=2^k$ elements has $\frac{k(k-1)}{2}$ "layers" where parallel compare-and-swap operations are used to rearrange a $k$ element vector into sorted order.

### Differentiable compare-and-swap

If we define the `softmax(a,b)` function (not the traditional "softmax" used for classification!) as the continuous approximation to the `max(a,b)` function:

$$\text{softmax}(a,b) = \log(e^a + e^b) \approx \max(a,b).$$

We can then fairly obviously write `softmin(a,b)` as:

$$\text{softmin}(a,b) = -\log(e^{-a} + e^{-b}) \approx \min(a,b).$$ More numerically stably we can write: 

$$\text{softmin}(a,b) = a + b - \text{softmax}(a,b).$$

These functions obviously aren't equal to max and min, but are relatively close, and differentiable. Note that we now have a differentiable compare-and-swap operation:

$$\text{high} = \text{softmax}(a,b), \text{low} = \text{softmin}(a,b), \text{where } \text{low}\leq \text{high}$$

Alternatively, we can use: 
$$\text{smoothmax}(a,b) = \frac{a (e^{\alpha a}) + b (e^{\alpha b})}{e^{\alpha a}+e^{\alpha b}}  \approx \max(a,b).$$  This has an adjustable smoothness parameter $\alpha$, with exact maximum as $\alpha \rightarrow \infty$ and pure averaging as $\alpha \rightarrow 0$.

## Differentiable sorting

For each layer in the sorting network, we can split all of the pairwise comparison-and-swaps into left-hand and right-hand sides which can be done simultaneously. We can any write function that selects the relevant elements of the vector as a multiply with a binary matrix.

For each layer, we can derive two binary matrices $L \in \mathbb{R}^{n \times \frac{n}{2}}$ and $R \in \mathbb{R}^{n \times \frac{n}{2}}$ which select the elements to be compared for the left and right hands respectively. This will result in the comparison between two $\frac{k}{2}$ length vectors. We can also derive two matrices $L' \in \mathbb{R}^{\frac{n}{2} \times n}$ and $R' \in \mathbb{R}^{\frac{n}{2} \times n}$ which put the results of the compare-and-swap operation back into the right positions.

Then, each layer $i$ of the sorting process is just:
$${\bf x}_{i+1} = L'_i[\text{softmin}(L_i{\bf x_i}, R_i{\bf x_i})] + R'_i[\text{softmax}(L_i{\bf x_i}, R_i{\bf x_i})]$$
$$ = L'_i\left(-\log\left(e^{-L_i{\bf x}_i} + e^{-R_i{\bf x}_i}\right)\right) +  R'_i\left(\log\left(e^{L_i{\bf x}_i} + e^{R_i{\bf x}_i}\right)\right)$$
which is clearly differentiable (though not very numerically stable -- the usable range of elements $x$ is quite limited in single float precision).

All that remains is to compute the matrices $L_i, R_i, L'_i, R'_i$ for each of the layers of the network. 

This process is excessively computation heavy, but easy to compute. We could also simplify this into two matrix multiplies, at the cost of a vector split and join in the middle (see the `woven` form later in this text). 

## Example

To sort four elements, we have a network like:

    0  1  2  3  
    ┕>>┙  │  │  
    │  │  ┕<<┙  
    ┕>>>>>┙  │  
    │  │  │  │  
    ┕>>┙  │  │  
    │  │  ┕>>┙  
    
This is equivalent to: 

    x[0], x[1] = cswap(x[0], x[1])
    x[3], x[2] = cswap(x[2], x[3])
    x[0], x[2] = cswap(x[0], x[2])
    x[0], x[1] = cswap(x[0], x[1])
    x[2], x[3] = cswap(x[2], x[3])
    
where `cswap(a,b) = (min(a,b), max(a,b))`

Replacing the indexing with matrix multiplies and `cswap` with a `softcswap = (softmin(a,b), softmax(a,b))` we then have the differentiable form.



# Test functions

In [1]:
from bitonic_tests import bitonic_network, pretty_bitonic_network

def neat_vec(n):
    # print a vector neatly    
    return "\t".join([f"{x:.2f}" for x in n])

# this should match the diagram at the top of the notebook
bitonic_network(16)

 0>1	 2<3	 4>5	 6<7	 8>9	10<11	12>13	14<15	
----------------------------------------------------------------
 0>2	 1>3	 4<6	 5<7	 8>10	 9>11	12<14	13<15	
 0>1	 2>3	 4<5	 6<7	 8>9	10>11	12<13	14<15	
----------------------------------------------------------------
 0>4	 1>5	 2>6	 3>7	 8<12	 9<13	10<14	11<15	
 0>2	 1>3	 4>6	 5>7	 8<10	 9<11	12<14	13<15	
 0>1	 2>3	 4>5	 6>7	 8<9	10<11	12<13	14<15	
----------------------------------------------------------------
 0>8	 1>9	 2>10	 3>11	 4>12	 5>13	 6>14	 7>15	
 0>4	 1>5	 2>6	 3>7	 8>12	 9>13	10>14	11>15	
 0>2	 1>3	 4>6	 5>7	 8>10	 9>11	12>14	13>15	
 0>1	 2>3	 4>5	 6>7	 8>9	10>11	12>13	14>15	
----------------------------------------------------------------


In [2]:
pretty_bitonic_network(16)

 0  1  2  3  4  5  6  7  8  9  10 11 12 13 14 15
 ╭──╯  │  │  │  │  │  │  │  │  │  │  │  │  │  │ 
 │  │  ╰──╮  │  │  │  │  │  │  │  │  │  │  │  │ 
 │  │  │  │  ╭──╯  │  │  │  │  │  │  │  │  │  │ 
 │  │  │  │  │  │  ╰──╮  │  │  │  │  │  │  │  │ 
 │  │  │  │  │  │  │  │  ╭──╯  │  │  │  │  │  │ 
 │  │  │  │  │  │  │  │  │  │  ╰──╮  │  │  │  │ 
 │  │  │  │  │  │  │  │  │  │  │  │  ╭──╯  │  │ 
 │  │  │  │  │  │  │  │  │  │  │  │  │  │  ╰──╮ 
 ╭─────╯  │  │  │  │  │  │  │  │  │  │  │  │  │ 
 │  ╭─────╯  │  │  │  │  │  │  │  │  │  │  │  │ 
 │  │  │  │  ╰─────╮  │  │  │  │  │  │  │  │  │ 
 │  │  │  │  │  ╰─────╮  │  │  │  │  │  │  │  │ 
 │  │  │  │  │  │  │  │  ╭─────╯  │  │  │  │  │ 
 │  │  │  │  │  │  │  │  │  ╭─────╯  │  │  │  │ 
 │  │  │  │  │  │  │  │  │  │  │  │  ╰─────╮  │ 
 │  │  │  │  │  │  │  │  │  │  │  │  │  ╰─────╮ 
 ╭──╯  │  │  │  │  │  │  │  │  │  │  │  │  │  │ 
 │  │  ╭──╯  │  │  │  │  │  │  │  │  │  │  │  │ 
 │  │  │  │  ╰──╮  │  │  │  │  │  │  │  │  │  │ 
 │  │  │  │  │  │  ╰

# Vectorised functions

## Testing

In [3]:
%load_ext autoreload
%autoreload 2

In [5]:
# Test sorting
import autograd.numpy as np # we can use plain numpy as well (but can't take grad!)


from differentiable_sorting import bitonic_matrices, diff_bisort, diff_argsort, bisort, smoothmax_bisort
matrices = bitonic_matrices(8)

# test bitonic sorting with exact maximum
for i in range(10):
    # these should all be in sorted order
    test = np.random.randint(0, 200, 8)
    print(bisort(matrices, test))    

[ 12.  43.  43.  86. 126. 167. 178. 188.]
[ 14.  35.  45.  69. 120. 132. 140. 157.]
[ 21.  26.  64.  80. 155. 158. 158. 173.]
[ 46.  82. 122. 123. 133. 146. 169. 190.]
[  2.  36.  77.  85. 118. 124. 125. 149.]
[ 54.  70.  71.  77. 107. 115. 128. 191.]
[ 22.  36.  95. 104. 117. 159. 166. 169.]
[ 28.  53.  72.  73.  75.  77. 159. 173.]
[ 32.  59.  70.  78.  86. 122. 130. 182.]
[  2.  10.  14.  27.  41.  69. 110. 136.]


In [6]:
for i in range(1, 11):
    k = 2**i
    matrices = bitonic_matrices(k)
    print(f"Testing sorting for {k} elements")
    for j in range(100):
        test = np.random.randint(0, 200, k)

        assert (np.allclose(bisort(matrices, test), np.sort(test)))

Testing sorting for 2 elements
Testing sorting for 4 elements
Testing sorting for 8 elements
Testing sorting for 16 elements
Testing sorting for 32 elements
Testing sorting for 64 elements
Testing sorting for 128 elements
Testing sorting for 256 elements
Testing sorting for 512 elements
Testing sorting for 1024 elements


## Differentiable sorting test

In [8]:
# Differentiable sorting 
np.set_printoptions(precision=2)
matrices = bitonic_matrices(8) 


for i in range(10):
    test = np.random.randint(-200,200,8)
    print("Softmax sorting   ", neat_vec(diff_bisort(matrices, test)))
    print("Smoothmax sorting ", neat_vec(smoothmax_bisort(matrices, test)))
    print("Exact sorting     ", neat_vec(bisort(matrices, test)))
    
    print()

Softmax sorting    -197.00	-173.00	-78.00	-7.00	73.00	121.98	126.02	167.00
Smoothmax sorting  -197.00	-173.00	-78.00	-7.00	73.00	122.07	125.93	167.00
Exact sorting      -197.00	-173.00	-78.00	-7.00	73.00	122.00	126.00	167.00

Softmax sorting    -181.01	-175.99	-132.00	-16.00	11.00	17.00	90.00	140.00
Smoothmax sorting  -180.97	-176.03	-132.00	-16.00	11.01	16.99	90.00	140.00
Exact sorting      -181.00	-176.00	-132.00	-16.00	11.00	17.00	90.00	140.00

Softmax sorting    -86.00	-7.00	33.00	44.00	75.00	116.00	174.96	179.04
Smoothmax sorting  -86.00	-7.00	33.00	44.00	75.00	116.00	175.15	178.85
Exact sorting      -86.00	-7.00	33.00	44.00	75.00	116.00	175.00	179.00

Softmax sorting    -179.00	-159.00	-132.00	-95.69	-94.31	-86.00	-28.00	165.00
Smoothmax sorting  -179.00	-159.00	-132.00	-95.00	-95.00	-86.00	-28.00	165.00
Exact sorting      -179.00	-159.00	-132.00	-95.00	-95.00	-86.00	-28.00	165.00

Softmax sorting    -121.00	-113.00	28.00	93.00	104.00	131.00	158.00	171.00
Smoothmax sorting  -120.

# Relaxed sorting
We can define a slighly modified function which interpolates between `softmax(a,b)` and `mean(a,b)`. The result is a sorting function that can be relaxed from sorting to averaging.

In [9]:
from differentiable_sorting import diff_bisort_smooth
# Differentiable smoothed sorting 
test = np.random.randint(-200,200,8)
print(f"Mean {np.mean(test):.2f}")
print()
print("Exact sorting            ", neat_vec(bisort(matrices, test)))
print()
for smooth in np.linspace(0, 1, 8):    
    print(f"Softmax.   smooth[{smooth:.2f}]  ", neat_vec(diff_bisort_smooth(matrices, test, smooth)))
    # smoothmax's alpha is the inverse of diff_bisort_smooth
    print(f"Smoothmax. alpha=[{1-smooth:.2f}]  ", neat_vec(smoothmax_bisort(matrices, test, alpha=1-smooth)))
    print()

Mean 58.25

Exact sorting             -154.00	22.00	42.00	65.00	95.00	107.00	134.00	155.00

Softmax.   smooth[0.00]   -154.00	22.00	42.00	65.00	95.00	107.00	134.00	155.00
Smoothmax. alpha=[1.00]   -154.00	22.00	42.00	65.00	95.00	107.00	134.00	155.00

Softmax.   smooth[0.14]   -65.55	32.85	43.40	61.36	82.65	85.94	107.51	117.84
Smoothmax. alpha=[0.86]   -154.00	22.00	42.00	65.00	95.00	107.00	134.00	155.00

Softmax.   smooth[0.29]   -7.43	42.36	47.26	59.29	69.30	76.17	86.97	92.08
Smoothmax. alpha=[0.71]   -154.00	22.00	42.00	65.00	95.00	107.00	134.00	155.00

Softmax.   smooth[0.43]   27.35	50.08	51.19	57.76	62.13	65.82	74.00	77.67
Smoothmax. alpha=[0.57]   -154.00	22.00	42.00	65.00	95.01	106.99	134.00	155.00

Softmax.   smooth[0.57]   45.19	52.23	54.33	57.18	59.95	62.15	66.39	68.58
Smoothmax. alpha=[0.43]   -154.00	22.00	42.00	65.00	95.07	106.93	134.00	154.99

Softmax.   smooth[0.71]   54.22	55.21	56.90	57.57	59.08	59.71	61.35	61.96
Smoothmax. alpha=[0.29]   -154.00	22.07	42.00	64.94	95.3

In [10]:
from autograd import jacobian
# show that we can take the derivative
jac_sort = jacobian(diff_bisort_smooth, argnum=1)
jac_sort(matrices, test, 0.05) # slight relaxation

array([[0.02, 0.86, 0.02, 0.02, 0.  , 0.02, 0.02, 0.02],
       [0.02, 0.02, 0.  , 0.  , 0.04, 0.86, 0.02, 0.02],
       [0.  , 0.02, 0.  , 0.02, 0.04, 0.02, 0.86, 0.02],
       [0.  , 0.  , 0.02, 0.  , 0.86, 0.05, 0.05, 0.02],
       [0.04, 0.02, 0.85, 0.02, 0.02, 0.  , 0.  , 0.03],
       [0.  , 0.02, 0.03, 0.02, 0.02, 0.02, 0.02, 0.85],
       [0.04, 0.02, 0.02, 0.86, 0.  , 0.  , 0.02, 0.02],
       [0.86, 0.02, 0.05, 0.05, 0.  , 0.02, 0.  , 0.  ]])

In [11]:
# show that we can take the derivative, applying some smoothing to get reasonable values
jac_sort = jacobian(smoothmax_bisort, argnum=1)
print(smoothmax_bisort(matrices, test, 0.2))
jac_sort(matrices, test, 0.2) 

[-154.     22.36   42.11   64.6    95.93  106.12  134.51  154.36]


array([[ 6.22e-15,  1.00e+00,  2.65e-15,  6.23e-15, -5.93e-17, -1.84e-14,
         3.34e-15, -8.90e-16],
       [-1.31e-09, -1.64e-14,  1.82e-05, -5.05e-08, -1.26e-03,  1.05e+00,
        -5.31e-02,  3.21e-06],
       [-2.08e-08, -3.83e-16,  4.24e-04, -1.36e-06, -7.60e-02, -5.30e-02,
         1.13e+00,  2.70e-05],
       [ 8.33e-07,  1.32e-17, -1.23e-02,  3.42e-05,  1.09e+00, -1.36e-03,
        -7.59e-02, -1.67e-03],
       [-2.44e-04, -1.26e-20,  1.12e+00, -1.13e-03, -1.34e-02,  1.87e-05,
         4.60e-04, -1.02e-01],
       [ 9.70e-04, -7.09e-23, -1.01e-01, -2.14e-02, -6.26e-04,  1.28e-07,
        -9.06e-06,  1.12e+00],
       [-1.01e-01,  1.54e-23, -3.11e-03,  1.12e+00,  3.26e-05, -2.34e-08,
        -7.69e-07, -1.97e-02],
       [ 1.10e+00,  3.02e-25,  1.31e-04, -1.01e-01, -6.21e-07, -2.64e-10,
         5.00e-09,  9.29e-04]])

## Woven form
We can "weave" the four matrices into two matrices for fewer multiplies at the cost of having to split and join the matrices at each layer.

In [14]:
from differentiable_sorting import bitonic_woven_matrices, dsort_weave

woven_matrices = bitonic_woven_matrices(8)

print("Exact sorting       ", neat_vec(bisort(matrices, test)))
print(f"Diff. (std.)       ", neat_vec(diff_bisort(matrices, test)))
print(f"Diff. (woven)      ", neat_vec(dsort_weave(woven_matrices, test)))
        

Exact sorting        -154.00	22.00	42.00	65.00	95.00	107.00	134.00	155.00
Diff. (std.)        -154.00	22.00	42.00	65.00	95.00	107.00	134.00	155.00
Diff. (woven)       -154.00	22.00	42.00	65.00	95.00	107.00	134.00	155.00


## Differentiable ranking / argsort
We can use a differentiable similarity measure between the input and output of the vector, e.g. an RBF kernel. We can use this to generate a normalised similarity matrix and apply this to a vector `[1, 2, 3, ..., n]`. This gives a differentiable ranking function.

As `sigma` gets larger, the result converges to giving all values the mean rank; as it goes to zero the result converges to the true rank.

In [15]:
from differentiable_sorting import order_matrix, diff_argsort

In [16]:
matrices = bitonic_matrices(8)

In [24]:
x = [5.0, -1.0, 9.5, 13.2, 16.2, 10.5, 42.0, 18.0]
np.set_printoptions(suppress=True)
print(x)
# show argsort
ranks = diff_argsort(matrices, x, sigma=0.5)
print(neat_vec(ranks))
print(np.argsort(ranks))

[5.0, -1.0, 9.5, 13.2, 16.2, 10.5, 42.0, 18.0]
1.00	0.00	2.05	4.00	5.00	2.97	7.00	6.00
[1 0 2 5 3 4 7 6]


In [26]:
# we now have differentiable argmax and argmin by indexing the rank vector
print(np.argmin(x), int(ranks[0]+0.5))
print(np.argmax(x), int(ranks[-1]+0.5))

1 1
6 6


In [27]:
print("Smoothed ranks")
test = x
for sigma in [0.1, 1, 10, 100, 1000]:     
    ranks = diff_argsort(matrices, test, sigma=sigma) 
    print(f"sigma={sigma:7.1f}  |", neat_vec(ranks))

Smoothed ranks
sigma=    0.1  | 1.00	0.00	2.00	4.00	5.00	3.00	7.00	6.00
sigma=    1.0  | 1.00	0.00	2.33	3.97	5.12	2.73	7.00	5.85
sigma=   10.0  | 2.55	1.92	3.01	3.38	3.65	3.11	6.79	3.82
sigma=  100.0  | 3.47	3.45	3.48	3.49	3.49	3.48	3.56	3.50
sigma= 1000.0  | 3.50	3.50	3.50	3.50	3.50	3.50	3.50	3.50


In [28]:
np.set_printoptions(precision=3)
jac_rank = jacobian(diff_argsort, argnum=1)
print(jac_rank(matrices, np.array(test), 1.0) )

[[ 0.001 -0.    -0.001 -0.    -0.    -0.    -0.    -0.   ]
 [-0.     0.    -0.    -0.    -0.    -0.    -0.    -0.   ]
 [-0.003 -0.     0.233 -0.022 -0.002 -0.206 -0.    -0.   ]
 [-0.001 -0.    -0.031  0.146 -0.033 -0.076 -0.    -0.005]
 [-0.    -0.    -0.002 -0.032  0.224 -0.002 -0.    -0.188]
 [-0.003 -0.    -0.209 -0.066 -0.004  0.283 -0.    -0.001]
 [-0.    -0.    -0.    -0.    -0.    -0.     0.    -0.   ]
 [-0.    -0.    -0.001 -0.013 -0.191 -0.002 -0.     0.207]]


# PyTorch example
We can verify that this is both parallelisable on the GPU and fully differentiable.

In [29]:
import torch
import numpy as np
from torch.autograd import Variable
import torch.nn.functional as F
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', device)

Device: cuda:0


In [42]:
from differentiable_sorting_torch import softmax, diff_argsort
from differentiable_sorting import dsort
matrices = bitonic_matrices(16)
torch_matrices = [[torch.from_numpy(matrix).float().to(device) for matrix in matrix_set] for matrix_set in matrices]


In [43]:
test_input = np.random.normal(0, 5, 16)
var_test_input = Variable(torch.from_numpy(test_input).float().to(device),
                          requires_grad=True)

result = dsort(torch_matrices, var_test_input, softmax=softmax)

# compute the Jacobian of the sorting function, to show we can differentiate through the
# sorting function
jac = []
for i in range(len(result)):
    jac.append(
        torch.autograd.grad(result[i], var_test_input, retain_graph=True)[0])

# 16 x 16 jacobian of the sorting matrix
print(torch.stack(jac))

tensor([[3.4062e-04, 8.7856e-04, 5.7044e-08, 2.7916e-03, 4.4100e-05, 5.3073e-06,
         2.1595e-04, 3.3469e-09, 1.3208e-02, 2.3107e-02, 4.4703e-06, 9.5774e-01,
         1.0295e-03, 1.1822e-05, 7.6105e-05, 5.4256e-04],
        [1.9060e-02, 4.5054e-02, 2.9057e-06, 1.8370e-01, 2.3067e-03, 2.4358e-04,
         9.4737e-03, 1.6484e-07, 1.9522e-01, 4.7163e-01, 9.1683e-05, 2.8573e-02,
         3.0151e-02, 2.7869e-04, 2.0594e-03, 1.2156e-02],
        [2.9231e-02, 9.3393e-02, 4.8280e-06, 2.5603e-01, 3.2653e-03, 4.9231e-04,
         1.9217e-02, 3.0734e-07, 2.9183e-01, 1.9657e-01, 2.1410e-04, 5.8898e-03,
         6.5825e-02, 5.6950e-04, 3.6078e-03, 3.3849e-02],
        [2.9448e-02, 9.3757e-02, 4.8280e-06, 2.5964e-01, 3.2800e-03, 4.9229e-04,
         1.9180e-02, 3.0734e-07, 2.8856e-01, 1.9640e-01, 2.1227e-04, 5.8859e-03,
         6.5418e-02, 5.6507e-04, 3.5841e-03, 3.3572e-02],
        [1.0592e-01, 2.0078e-01, 2.1935e-05, 9.0817e-02, 2.7345e-02, 2.9336e-03,
         1.2352e-01, 1.6764e-06, 6.5530

In [45]:
result = diff_argsort(torch_matrices, var_test_input)
print(result)


tensor([ 7.0004,  5.0000, 14.0000,  3.0000,  9.0000, 12.0000,  7.0001, 15.0000,
         2.1415,  2.0000, 12.0000,  0.0000,  5.0065, 11.0046,  9.8874,  6.0000],
       device='cuda:0', grad_fn=<MvBackward>)
